In [86]:
import os
import gzip
from Bio import SeqIO
import re
from newick import read, dump

In [87]:
trees = read('./dataset/OG0000002_tree.txt')
node_needed = trees[0].get_node('n441').get_leaves()
all_node = trees[0].get_leaves()

In [88]:
trees[0].prune(node_needed, inverse=True)

In [74]:
c_elegans_remove = ['c_elegans_ref_protein_PAR-1_ isoform c', 
                   'c_elegans_H39E23.1l wormpep_CE48381 gene_WBGene00003916 locus_par-1 status_Partially_confirmed uniprot_Q9TW45 insdc_CCW45975.1 product_"Serine\/threonine-protein kinase par-1"',
                   'c_elegans_H39E23.1c wormpep_CE40085 gene_WBGene00003916 locus_par-1 status_Confirmed uniprot_Q9TW45 insdc_CAJ85756.1 product_"Serine\/threonine-protein kinase par-1"',
                   'c_elegans_H39E23.1g wormpep_CE45263 gene_WBGene00003916 locus_par-1 status_Confirmed uniprot_Q9TW45 insdc_CBW44387.1 product_"Serine\/threonine-protein kinase par-1"',
                   'c_elegans_H39E23.1f wormpep_CE45159 gene_WBGene00003916 locus_par-1 status_Confirmed uniprot_Q9TW45 insdc_CBO24859.1 product_"Serine\/threonine-protein kinase par-1"',
                   'c_elegans_ref_protein_PAR-1_ isoform g',
                   'c_elegans_ref_protein_PAR-1_ isoform f',
                   'c_elegans_ref_protein_PAR-1_ isoform j',
                   'c_elegans_ref_protein_PAR-1_ isoform b',
                   'c_elegans_H39E23.1b wormpep_CE27768 gene_WBGene00003916 locus_par-1 status_Partially_confirmed uniprot_Q9TW45 insdc_CAB54262.2 product_"Serine\/threonine-protein kinase par-1"',
                   'c_elegans_ref_protein_PAR-1_ isoform d',
                   'c_elegans_H39E23.1d wormpep_CE44808 gene_WBGene00003916 locus_par-1 status_Partially_confirmed uniprot_Q9TW45 insdc_CAQ16148.2 product_"Serine\/threonine-protein kinase par-1"',
                   'c_elegans_ref_protein_PAR-1_ isoform e',
                   'c_elegans_H39E23.1m wormpep_CE52350 gene_WBGene00003916 locus_par-1 status_Partially_confirmed uniprot_A0A2C9C376 insdc_SOF58805.1 product_"Serine\/threonine-protein kinase par-1"',
                   'c_elegans_H39E23.1e wormpep_CE44733 gene_WBGene00003916 locus_par-1 status_Partially_confirmed uniprot_Q9TW45 insdc_CBL43447.1 product_"Serine\/threonine-protein kinase par-1"',
                   'c_elegans_H39E23.1j wormpep_CE46555 gene_WBGene00003916 locus_par-1 status_Partially_confirmed uniprot_Q9TW45 insdc_CCE71394.1 product_"Serine\/threonine-protein kinase par-1"',
                   'c_elegans_H39E23.1a wormpep_CE23838 gene_WBGene00003916 locus_par-1 status_Confirmed uniprot_Q9TW45 insdc_CAB54263.1 product_"Serine\/threonine-protein kinase par-1"']
trees[0].prune_by_names(c_elegans_remove)

In [90]:
def clean_node(node_name):
    regex = r"[a-z]_[a-zA-Z]+_[a-zA-Z1-9]+.[a-zA-Z1-9]+.[a-zA-Z1-9]+.[a-zA-Z1-9]+"
    if re.search(regex, str(node_name)):
        node_name.name = re.search(regex, str(node_name)).group(0)
        
trees[0].visit(clean_node)

In [91]:
leaves = trees[0].get_leaves()

In [92]:
unique = {}

for node in leaves:
    regex = r"[a-z]_[a-zA-Z]+_[a-zA-Z1-9]"
    if re.search(regex, node.name):
        node_new_name = re.search(regex, str(node.name)).group(0)
    if node_new_name in unique:
        unique[node_new_name] = max(unique[node_new_name], node.length)
    else:
        unique[node_new_name] = node.length      

In [93]:
for node in leaves:
    regex = r"[a-z]_[a-zA-Z]+_[a-zA-Z1-9]"
    if re.search(regex, node.name):
        node_new_name = re.search(regex, str(node.name)).group(0)
    if node.length != unique[node_new_name]:
        trees[0].prune_by_names(node.name)

In [95]:
c_elegans_remove = ['c_elegans_ref_protein_PAR-1']
trees[0].prune_by_names(c_elegans_remove)

In [96]:
while trees[0].walk(mode='postorder'):
    atleast_once = True
    for n in trees[0].walk(mode='postorder'):
        regex = r"^n[1-9]+"
        if n.ancestor and len(n.descendants) == 0 and re.search(regex, n.name):
            trees[0].prune_by_names(n.name)
            atleast_once = False
    if atleast_once == True:
        break

In [97]:
trees[0].get_leaves()

[Node("c_quiockensis_CSP38.g3072.t2"),
 Node("c_inopinata_Sp34_503655"),
 Node("c_nigoni_Cni-par-1.1"),
 Node("c_panamensis_CSP28.g22133.t1 transcript"),
 Node("c_zanzibari_CSP26.g6511.t3 transcript"),
 Node("c_briggsae_CBG04756o wormpep_CBP46111"),
 Node("c_japonica_CJA32335 wormpep_JA61847 gene"),
 Node("c_elegans_H39E23.1m wormpep_CE5235"),
 Node("c_brenneri_CBN20317 wormpep_CN373"),
 Node("d_pachys_WR25_19141.1 transcript"),
 Node("d_coronatus_DCO_002853 transcript_DCO_002853 gene_DCO_002853"),
 Node("h_polygyrus_HPOL_0000412901-mRNA-1 transcript_HPOL_0000412901-mRNA-1 gene_HPOL_0000412901"),
 Node("o_tipulae_OTIPU.nOt.2"),
 Node("c_parvicauda_CSP21.g10579.t2"),
 Node("s_digitata_sdigi.contig38.g2562.t1"),
 Node("loa_loa_EN70_862 transcript_EN70_862 gene_EN70_862"),
 Node("a_viteae_nAv.1.0.1.t09244-RA transcript_nAv.1.0.1.t09244-RA gene_nAv.1.0.1.g09244"),
 Node("l_sigmodontis_nLs.2.1.2"),
 Node("o_volvulus_OVOC6199f wormpep_OVP14656 gene"),
 Node("d_immitis_nDi.2.2.2"),
 Node("b_m

In [98]:
with open('required_tree.txt', 'w') as fobj:
    dump(trees, fobj)